In [1]:
import os
from typing import List, Iterable, Tuple
import pandas as pd
import numpy as np
import datasets
from tqdm.notebook import tqdm
import random

from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
import evaluate
from sklearn.model_selection import GroupKFold

In [2]:
DATA_DIR = 'crisis_data'
FILE_BLACKLIST = [
    'crisis_data/Afera Rywina.xlsx',
    'crisis_data/Ministerstwo Zdrowia_respiratory od handlarza bronią.xlsx',
    'crisis_data/Fake news_baza publikacji.xlsx'
]

files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f[-5:] == '.xlsx']
for f in FILE_BLACKLIST:
    files.remove(f)

In [3]:
def load_dataset(filename: str) -> datasets.Dataset:
    df = pd.read_excel(filename)
    df = df[['Kryzys', 'Tytuł publikacji', 'Lead', 'Kontekst publikacji']]

    if df['Kryzys'].hasnans:
        df['Kryzys'] = df['Kryzys'].notna()
    else:
        df['Kryzys'] = (df['Kryzys'] != 'NIE') & (df['Kryzys'] != 'Nie')
    if df['Kryzys'].nunique() != 2:
        raise RuntimeError(f'Crisis column data error in file {filename}.')

    df['text'] = df.apply(lambda x: ".".join([str(x['Tytuł publikacji']), str(x['Lead']), str(x['Kontekst publikacji'])]), axis=1)
    df = df[['Kryzys', 'text']].rename(columns={'Kryzys': 'label'})
    return datasets.Dataset.from_pandas(df).cast_column('label', datasets.ClassLabel(names=['negative', 'positive']))

In [4]:
def prepare_dataset(filenames: List[str]) -> datasets.Dataset:
    dss = []
    for i, filename in enumerate(tqdm(filenames)):
        df = pd.read_excel(filename)
        df = df[['Kryzys', 'Tytuł publikacji', 'Lead', 'Kontekst publikacji']]

        if df['Kryzys'].hasnans:
            df['Kryzys'] = df['Kryzys'].notna()
        else:
            df['Kryzys'] = (df['Kryzys'] != 'NIE') & (df['Kryzys'] != 'Nie')
        if df['Kryzys'].nunique() != 2:
            raise RuntimeError(f'Crisis column data error in file {filename}.')

        df['text'] = df.apply(lambda x: ".".join([str(x['Tytuł publikacji']), str(x['Lead']), str(x['Kontekst publikacji'])]), axis=1)
        df = df[['Kryzys', 'text']].rename(columns={'Kryzys': 'label'})
        df['group'] = i
        dss.append(datasets.Dataset.from_pandas(df).cast_column('label', datasets.ClassLabel(names=['negative', 'positive'])))
    return datasets.concatenate_datasets(dss)

In [5]:
def fold_dataset(ds: datasets.Dataset, n_splits: int = 10) -> List[Tuple[datasets.Dataset, datasets.Dataset, datasets.Dataset]]:
    fold = GroupKFold(n_splits=n_splits)
    folds = list(fold.split(ds, groups=ds['group']))
    ds = ds.remove_columns('group')
    val_splits = [split[1] for split in folds[-1:] + folds[:-1]]
    splits = [(list(set(train_idx) - set(val_idx)), val_idx, test_idx) for ((train_idx, test_idx), val_idx) in zip(folds, val_splits)]
    return [(datasets.Dataset.from_dict(ds[train_idx]), datasets.Dataset.from_dict(ds[val_idx]), datasets.Dataset.from_dict(ds[test_idx])) for train_idx, val_idx, test_idx in splits]

def split_dataset(ds: datasets.Dataset, n_splits: int = 10, validation: bool = False) -> datasets.DatasetDict:
    fold = GroupKFold(n_splits=n_splits)
    folds = list(fold.split(ds, groups=ds['group']))
    ds = ds.remove_columns('group')
    dsd = datasets.DatasetDict()
    if validation:
        val_idx = folds[1][1]
        train_idx = list(set(folds[0][0]) - set(val_idx))
        test_idx = folds[0][1]
        dsd['train'] = datasets.Dataset.from_dict(ds[train_idx])
        dsd['val'] = datasets.Dataset.from_dict(ds[val_idx])
        dsd['test'] = datasets.Dataset.from_dict(ds[test_idx])
    else:
        train_idx, test_idx = folds[0]
        dsd['train'] = datasets.Dataset.from_dict(ds[train_idx])
        dsd['test'] = datasets.Dataset.from_dict(ds[test_idx])
    return dsd

In [6]:
# ds = prepare_dataset(files)
# ds.save_to_disk('other_data/text_dataset')

In [7]:
# dss = {}
# for fname in tqdm(files):
#     dss[fname[len(DATA_DIR)+1:-5]] = load_dataset(fname)
# dss = datasets.DatasetDict(dss)
# dss.save_to_disk('other_data/text_dataset')

In [8]:
# ds = datasets.load_from_disk('other_data/text_dataset')

In [9]:
tokenizer = AutoTokenizer.from_pretrained("sdadas/polish-distilroberta")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
# tokenized_ds = ds.map(preprocess_function, batched=True)
# tokenized_ds.save_to_disk('other_data/tokenized_dataset')

In [11]:
tokenized_ds = datasets.load_from_disk('other_data/tokenized_dataset').shuffle().select(range(20000))

In [12]:
tokenized_ds = split_dataset(tokenized_ds)

In [17]:
len(tokenized_ds['train'][2]['input_ids'])

98

In [13]:
model = AutoModelForSequenceClassification.from_pretrained("sdadas/polish-distilroberta", num_labels=2)

Some weights of the model checkpoint at sdadas/polish-distilroberta were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/polish-distilroberta and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier

In [14]:
def compute_metrics(eval_pred):
    load_accuracy = evaluate.load("accuracy")
    load_f1 = evaluate.load("f1")
    load_precision = evaluate.load("precision")
    load_recall = evaluate.load("recall")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    precision = load_precision.compute(predictions=predictions, references=labels)["precision"]
    recall = load_recall.compute(predictions=predictions, references=labels)["recall"]
    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [15]:
training_args = TrainingArguments(
    output_dir='other_data/trained',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/home/proto/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17111
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5350
  Number of trainable parameters = 81917186
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is fast

Step,Training Loss


Saving model checkpoint to other_data/trained/checkpoint-1070
Configuration saved in other_data/trained/checkpoint-1070/config.json
Configuration saved in other_data/trained/checkpoint-1070/config.json
Model weights saved in other_data/trained/checkpoint-1070/pytorch_model.bin
tokenizer config file saved in other_data/trained/checkpoint-1070/tokenizer_config.json
Special tokens file saved in other_data/trained/checkpoint-1070/special_tokens_map.json
Saving model checkpoint to other_data/trained/checkpoint-2140
Configuration saved in other_data/trained/checkpoint-2140/config.json
Model weights saved in other_data/trained/checkpoint-2140/pytorch_model.bin
tokenizer config file saved in other_data/trained/checkpoint-2140/tokenizer_config.json
Special tokens file saved in other_data/trained/checkpoint-2140/special_tokens_map.json
Saving model checkpoint to other_data/trained/checkpoint-3210
Configuration saved in other_data/trained/checkpoint-3210/config.json
Model weights saved in other_d

TrainOutput(global_step=5350, training_loss=0.09573714568236164, metrics={'train_runtime': 657.0449, 'train_samples_per_second': 130.212, 'train_steps_per_second': 8.143, 'total_flos': 7111032102527232.0, 'train_loss': 0.09573714568236164, 'epoch': 5.0})

In [17]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2889
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2889
  Batch size = 16


{'eval_loss': 0.21016447246074677,
 'eval_accuracy': 0.9567324333679473,
 'eval_f1': 0.06015037593984962,
 'eval_precision': 0.03225806451612903,
 'eval_recall': 0.4444444444444444,
 'eval_runtime': 12.3362,
 'eval_samples_per_second': 234.189,
 'eval_steps_per_second': 14.672,
 'epoch': 5.0}

In [18]:
trainer.save_model('other_data/text_classifier')

Saving model checkpoint to other_data/text_classifier
Configuration saved in other_data/text_classifier/config.json
Model weights saved in other_data/text_classifier/pytorch_model.bin
tokenizer config file saved in other_data/text_classifier/tokenizer_config.json
Special tokens file saved in other_data/text_classifier/special_tokens_map.json


In [19]:
classifier = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device='cuda:0')

In [20]:
classifier(['Sasin'])

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'label': 'LABEL_1', 'score': 0.9864873290061951}]

In [24]:
tokenized_ds

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
